In [87]:
from google.colab import drive

In [90]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/CEAT Dataset/sen_dic_1.pickle'  ./

# Dataset processing


In [1]:
import pickle

In [ ]:
dataset = pickle.load(open('/content/sen_dic_1.pickle','rb'))

In [ ]:
type(dataset)

dict

In [ ]:
len(dataset)

417

In [ ]:
sliced_items = list(dataset.items())[:10]

In [ ]:
print(len(sliced_items))

10


In [ ]:
sliced_items[0]

In [ ]:
len(dataset['Joan'])

10000

In [ ]:
for key, value in sliced_items:
    print(key, value)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
from itertools import islice

In [ ]:
for key, value in islice(dataset.items(), 100):
    print(key, value)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
sentence_list = []
word_list = []

for key, value in dataset.items():
    if (len(value) != 10000):
        print("Value not consistent")
    for sentence in value:
        sentence_list.append(sentence)
        word_list.append(key)

assert len(word_list) == len(sentence_list)
print("Sentence List length: ", len(sentence_list))

In [3]:
import pandas as pd

In [ ]:
df = pd.DataFrame({"Sentence": sentence_list, "Word": word_list}, index=range(1, len(sentence_list)+1))

In [ ]:
df.to_csv("CEAT_Dataset.csv")

In [ ]:
!cp ./CEAT_Dataset.csv '/content/drive/MyDrive/CEAT Dataset/'

In [ ]:
df.iloc[2]["Sentence"]

'My father used to tell me stories about my great grandfather opening this deli  1926  he immigrated from Germany to open Eckheart s Deli  Since then it s become something of a fixture in the neighborhood  due in no small part to the generations of Huber s staffing it  a straight line from my great grandfather to me \n\nWe make sandwiches  sell meat  butchery and coffee  We re not famous by any stretch  but we do have our regulars from around the neighborhood  Carol comes in for beef roast and pork tenderloin on wednesdays  usually about 2 pounds each  and a cup of coffee to drink while she tells anyone who will listen stories of her children  Jerry comes in around lunchtime once a week for a date with his wife  roast beef with mayo and a side of comments about his blood pressure  The oddest  though  is Ben  Ben s a good guy  always comes in on Tuesday and orders the house Reuben  Only thing we dont make ourselves for that  by the by  is the bread  and only because we got a deal with T

In [5]:
import re

In [ ]:
text = df.iloc[2]["Sentence"]

matches = re.findall(r'Amy', text)
print(matches)

['Amy']


## Bert embedding generating

In [6]:
!pip install transformers

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import torch

from transformers import BertModel, BertTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel,GPT2Model
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import logging
logging.getLogger('transformers.tokenization_utils').disabled = True
import numpy as np
import json
import pickle
import datetime
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import norm.sf as Survival_Function

In [ ]:
torch.cuda.is_available()

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-cased')
model_bert = BertModel.from_pretrained('bert-base-cased')
model_bert.eval()
if torch.cuda.is_available():
    model_bert.to('cuda')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_bert.__class__.__name__

'BertModel'

In [9]:
flowers = ['aster', 'clover', 'hyacinth', 'marigold', 'poppy', 'azalea', 'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil', 'lilac', 'pansy', 'tulip', 'buttercup', 'daisy', 'lily', 'peony', 'violet', 'carnation',
'magnolia', 'petunia', 'zinnia','gladiola']
insects = ['ant', 'caterpillar', 'flea', 'locust', 'spider', 'bedbug', 'centipede', 'fly', 'maggot', 'tarantula',
'bee', 'cockroach', 'gnat', 'mosquito', 'termite', 'beetle', 'cricket', 'hornet', 'moth', 'wasp',
'dragonfly', 'horsefly', 'roach', 'weevil','blackfly']
pleasant = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure', 'diamond', 'gentle', 'honest',
'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise', 'family',
'happy', 'laughter', 'paradise', 'vacation']
unpleasant = ['abuse', 'crash', 'filth', 'murder', 'sickness', 'accident', 'death', 'grief', 'poison', 'stink',
'assault', 'disaster', 'hatred', 'pollute', 'tragedy', 'divorce', 'jail', 'poverty', 'ugly', 'cancer', 'kill', 'rotten',
'vomit', 'agony', 'prison']

In [10]:
def short_sen(sen,wd):
    wds = sen.split()
    wd_idx = wds.index(wd)
    if len(wds) >=9:
        if wd_idx < 4:
            wds_used = wds[:9]
        elif (len(wds) - wd_idx - 1 < 4):
            wds_used = wds[-9:]
        else:
            wds_used = wds[(wd_idx-4):(wd_idx+4)]
        new_sen = ' '.join(wds_used)
    else:
        new_sen = sen
    return new_sen

In [ ]:
sen = "And the curtains would fall on this great act of life when the mesmerizing dance of order shall fall."
wd = "curtains"
short_sen(sen, wd)

'And the curtains would fall on this great act'

In [11]:
wd_lst = flowers + insects + pleasant + unpleasant
wd_idx_dict = {wd:[] for wd in wd_lst}
out_dict = {wd:[] for wd in wd_lst}
out_dict

{'aster': [],
 'clover': [],
 'hyacinth': [],
 'marigold': [],
 'poppy': [],
 'azalea': [],
 'crocus': [],
 'iris': [],
 'orchid': [],
 'rose': [],
 'bluebell': [],
 'daffodil': [],
 'lilac': [],
 'pansy': [],
 'tulip': [],
 'buttercup': [],
 'daisy': [],
 'lily': [],
 'peony': [],
 'violet': [],
 'carnation': [],
 'magnolia': [],
 'petunia': [],
 'zinnia': [],
 'gladiola': [],
 'ant': [],
 'caterpillar': [],
 'flea': [],
 'locust': [],
 'spider': [],
 'bedbug': [],
 'centipede': [],
 'fly': [],
 'maggot': [],
 'tarantula': [],
 'bee': [],
 'cockroach': [],
 'gnat': [],
 'mosquito': [],
 'termite': [],
 'beetle': [],
 'cricket': [],
 'hornet': [],
 'moth': [],
 'wasp': [],
 'dragonfly': [],
 'horsefly': [],
 'roach': [],
 'weevil': [],
 'blackfly': [],
 'caress': [],
 'freedom': [],
 'health': [],
 'love': [],
 'peace': [],
 'cheer': [],
 'friend': [],
 'heaven': [],
 'loyal': [],
 'pleasure': [],
 'diamond': [],
 'gentle': [],
 'honest': [],
 'lucky': [],
 'rainbow': [],
 'diploma': [

In [ ]:
sen_dict = pickle.load(open('sen_dic_1.pickle','rb'))
for wd in wd_lst[:10]:
    print("*******************", wd)
    for idx,sen in enumerate(sen_dict[wd]):
        if idx == 5:
            break

        sen = short_sen(sen,wd)
        print(sen)

******************* aster
ooc i distinctly remember aster giving zach something
Aster Vackert   u aster vackert 
goto nearby aster mediclinic and ask for pollution masks
Nice try   M  aster  r  ace Snapping Turtle 
have an ivy and aster dress but it
******************* clover
like a 4 leaf clover some pretty indian
http cinemaknifefight files wordpress com 2011 06 clover jpg
Eight leaf clover from Futurama the Subtle Knife Excalibur
Would you recommend clover hooks And where do you
Got clover hooks for Christmas I absolutely love them
******************* hyacinth
perfumed by jasmine and hyacinth past street vendors
re muscari aka grape hyacinth Standard hyacinth have
it s muscari  also called grape hyacinth 
also sometimes called grape hyacinth might be a
It s not hyacinth It s muscari sometimes called
******************* marigold
for a recent movie the best exotic marigold hotel
My mum went to marigold 20 30 yrs
hello palette Nyx yellow marigold with some random
o clock lavender chrys

In [ ]:
enc = tokenizer_bert.encode('[MASK] is in the sky',add_special_tokens=False)
enc

[103, 1110, 1107, 1103, 3901]

In [ ]:
torch.tensor(enc)

tensor([ 103, 1110, 1107, 1103, 3901])

In [ ]:
torch.tensor(enc).unsqueeze(0)

tensor([[ 103, 1110, 1107, 1103, 3901]])

In [ ]:
torch.tensor(enc).unsqueeze(0).tolist()[0]

[103, 1110, 1107, 1103, 3901]

In [ ]:
input_ids = torch.tensor(enc).unsqueeze(0)
input_ids

tensor([[ 103, 1110, 1107, 1103, 3901]])

In [ ]:
outputs = model_bert(input_ids)
outputs

In [12]:
import random

In [13]:
def getSentencesSample(sentenceList, maxItems = 1000):
    numItems = min(maxItems, len(sentenceList))
    items = sentenceList if numItems < maxItems else random.sample(sentenceList, maxItems)
    return items

In [14]:
def shortenSentence(sentence, word, length = 9):
    words = sentence.split()
    wd_idx = words.index(word)
    if len(words) >=9:
        if wd_idx < 4:
            wds_used = words[:9]
        elif (len(words) - wd_idx - 1 < 4):
            wds_used = words[-9:]
        else:
            wds_used = words[(wd_idx-4):(wd_idx+4)]
        newSentence = ' '.join(wds_used)
    else:
        newSentence = sentence
    return newSentence

In [15]:
def getEmbeddingsFromSent(tokenizer, model, wordList, sentenceDict, dump = "pickle", maxSentenceNum = 1000):
    wordIndexDict = {}
    wordEmbeddingDict = {word: [] for word in wordList}

    for word in wordList:
        wordIndexDict[word] = torch.tensor(tokenizer.encode(word, add_special_tokens=False)).unsqueeze(0).tolist()[0]

    for word in wordList:
        targetIndex = wordIndexDict[word]
        sentenceList = getSentencesSample(sentenceDict[word], maxSentenceNum)

        for index, Sentence in tqdm(enumerate(sentenceList), desc="Processing Sentences", total=maxSentenceNum):
            Sentence = shortenSentence(Sentence, word)
            inputIds = torch.tensor(tokenizer.encode(Sentence, add_special_tokens=False)).unsqueeze(0)
            if torch.cuda.is_available():
                inputIds = inputIds.to('cuda')
            targetIndexInSent = inputIds.tolist()[0].index(targetIndex)
            outputs = model(inputIds)
            exact_state_vector = outputs[0][0,int(targetIndexInSent),:].cpu().detach().numpy()
            wordEmbeddingDict[word].append(exact_state_vector)
        wordEmbeddingDict[word] = np.array(wordEmbeddingDict[word])

    if dump == "pickle":
        name = model.__class__.__name__ + "_embs_" + sentenceDict["__name__"] + ".pickle"
        pickle.dump(wordEmbeddingDict,open(name,'wb'))
    else:
        return wordEmbeddingDict

In [16]:
def bert(wd_lst,out_name):
    sen_dict = pickle.load(open('sen_dic_1.pickle','rb'))
    wd_idx_dict = {wd:[] for wd in wd_lst}
    out_dict = {wd:[] for wd in wd_lst}
    for wd in wd_lst:
        current_idx = torch.tensor(tokenizer_bert.encode(wd,add_special_tokens=False)).unsqueeze(0).tolist()[0]
        wd_idx_dict[wd] = current_idx

    i = 0
    for wd in wd_lst:
        target = wd_idx_dict[wd][-1]
        tem = []
        for idx,sen in enumerate(sen_dict[wd]):
            i += 1
            if i%5000 == 0:
                now = datetime.datetime.now()
                print(now.strftime("%Y-%m-%d %H:%M:%S"))
                print(str(i)+' finished.')
            if idx == 1000:
                break

            sen = short_sen(sen,wd)
            input_ids = torch.tensor(tokenizer_bert.encode(sen, add_special_tokens=False)).unsqueeze(0)
            input_ids = input_ids.to('cuda')
            exact_idx = input_ids.tolist()[0].index(target)
            outputs = model_bert(input_ids)
            exact_state_vector = outputs[0][0,int(exact_idx),:].cpu().detach().numpy()
            out_dict[wd].append(exact_state_vector)
    n = 'bert_'+out_name+'.pickle'
    pickle.dump(out_dict,open(n,'wb'))

In [ ]:
lst = flowers + insects + pleasant + unpleasant

bert(lst,'weat1')
print("bert finish")

## CEAT testing

In [20]:
def associate(w,A,B):
    return cosine_similarity(w.reshape(1,-1),A).mean() - cosine_similarity(w.reshape(1,-1),B).mean()

In [21]:
def EffectSize(X,Y,A,B):
    delta_mean =  np.mean([associate(X[i,:],A,B) for i in range(X.shape[0])]) - np.mean([associate(Y[i,:],A,B) for i in range(Y.shape[0])])

    XY = np.concatenate((X,Y),axis=0)
    s = [associate(XY[i,:],A,B) for i in range(XY.shape[0])]

    std_dev = np.std(s,ddof=1)
    var = std_dev**2

    return delta_mean/std_dev, var

In [22]:
def CEAT_DataGeneration(weatGroups: list, embeddingsFileName: str, testSet: int, nSample = 10000, seed=32, model="bert", experimentType = "random", save = False):
    np.random.seed(seed=seed)
    embeddingsDict = pickle.load(open(embeddingsFileName, 'rb'))

    effectSizeArray = np.array([], dtype = np.float32)
    varianceArray = np.array([], dtype = np.float32)

    weatGroup = weatGroups[testSet]
    XSet = weatGroup[0]
    YSet = weatGroup[1]
    ASet = weatGroup[2]
    BSet = weatGroup[3]

    for i in range(nSample):

        X = np.array(embeddingsDict[word][np.random.randint(0, len(embeddingsDict[word]))] for word in XSet)
        Y = np.array(embeddingsDict[word][np.random.randint(0, len(embeddingsDict[word]))] for word in YSet)
        A = np.array(embeddingsDict[word][np.random.randint(0, len(embeddingsDict[word]))] for word in ASet)
        B = np.array(embeddingsDict[word][np.random.randint(0, len(embeddingsDict[word]))] for word in BSet)

        effectSize, variance = EffectSize(X, Y, A, B)

        effectSizeArray = np.append(effectSizeArray, effectSize)
        varianceArray = np.append(varianceArray, variance)

    if save:
        pickle.dump(effectSizeArray, open("es_"+model+".pickle", "wb"))
        pickle.dump(variance, open("var_"+model+".pickle", "wb"))

    return effectSizeArray, varianceArray

In [ ]:
def CEAT_MetaAnalysis(effectSizeArray, V, nSample = 10000):
    W = 1 / V
    Q = np.sum(W*(effectSizeArray**2)) - ((np.sum(W*effectSizeArray)**2) / np.sum(W))

    df = nSample - 1;

    if Q >= df:
        C = np.sum(W) -np.sum(W**2)/np.sum(W)
        sigma_square_btn = (Q - df)/C
    else:
        sigma_square_btn = 0

    v = 1 / (V + sigma_square_btn)

    CES = np.sum(v * effectSizeArray) / np.sum(v)

    SE_CES = np.sqrt(1 / np.sum(v))

    p_value = Survival_Function((CES/SE_CES), loc = 0, scale = 1)

    return CES, p_value

## Stemming and preprocessing


In [23]:
!git clone https://github.com/abhik1505040/bengali-stemmer.git

Cloning into 'bengali-stemmer'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 101 (delta 4), reused 3 (delta 3), pack-reused 95 (from 1)
Receiving objects: 100% (101/101), 25.36 KiB | 2.54 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [25]:
%cd bengali-stemmer/
!pip install .
%cd ../

/content/bengali-stemmer
Processing /content/bengali-stemmer
  Preparing metadata (setup.py) ... done
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6390 sha256=f85c19a4501d92ebc9c623ba537077c263f8665388f7f42b2ef4e781f08d350e
  Stored in directory: /root/.cache/pip/wheels/cc/ff/d0/2f0a6b6a6671f4661e32e9fe347bbc3dd6db1d296dc95ad1ed
Successfully built bengali-stemmer
/content


In [26]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
import os

In [28]:
parser = RafiStemmer()

In [29]:
print(parser.groups)

[['ই', 'ও', 'তো'], ['কে', 'তে', 'রা'], ['চ্ছি', 'চ্ছিল', 'চ্ছে', 'চ্ছিস', 'চ্ছিলেন', 'চ্ছ', 'য়েছে', 'েছ', 'েছে', 'েছেন', 'রছ', 'রব', 'েল', 'েলো', 'ওয়া', 'েয়ে', 'য়', 'য়ে', 'য়েছিল', 'েয়েছিল', 'েছিল', 'েয়েছিলেন', 'ে.েছিলেন', 'েছিলেন', 'লেন', 'দের', 'ে.ে', 'ের', 'ার', 'েন', 'বেন', 'িস', 'ছিস', 'ছিলি', 'ছি', 'ছে', 'লি', 'বি', 'ে'], ['টি', 'টির', 'েরটা', 'েরটার', 'টা', 'টার', 'গুলো', 'গুলোর', 'েরগুলো', 'েরগুলোর']]


In [30]:
print(parser.replace_rules)

{'রছ': 'র', 'রব': 'র', 'েয়ে': 'া', 'েয়েছিল': 'া', 'েয়েছিলেন': 'া', 'ে.েছিলেন': 'া.', 'ে.ে': 'া.'}


In [31]:
stemmer = RafiStemmer()

In [32]:
stemmer.groups

[['ই', 'ও', 'তো'],
 ['কে', 'তে', 'রা'],
 ['চ্ছি',
  'চ্ছিল',
  'চ্ছে',
  'চ্ছিস',
  'চ্ছিলেন',
  'চ্ছ',
  'য়েছে',
  'েছ',
  'েছে',
  'েছেন',
  'রছ',
  'রব',
  'েল',
  'েলো',
  'ওয়া',
  'েয়ে',
  'য়',
  'য়ে',
  'য়েছিল',
  'েয়েছিল',
  'েছিল',
  'েয়েছিলেন',
  'ে.েছিলেন',
  'েছিলেন',
  'লেন',
  'দের',
  'ে.ে',
  'ের',
  'ার',
  'েন',
  'বেন',
  'িস',
  'ছিস',
  'ছিলি',
  'ছি',
  'ছে',
  'লি',
  'বি',
  'ে'],
 ['টি',
  'টির',
  'েরটা',
  'েরটার',
  'টা',
  'টার',
  'গুলো',
  'গুলোর',
  'েরগুলো',
  'েরগুলোর']]

In [ ]:
stemmer.stem_word('উপলব্ধিতে')

'উপলব্ধি'

In [ ]:
stemmer.stem_word('চমৎকার')

'চমৎক'

In [33]:
from bengali_stemmer.rafikamal2014 import RafiStemmer

In [ ]:
stemmer = RafiStemmer()
stemmer.stem_word('সে কারণেই পর্যটকরা কুয়াকাটায় এসে প্রকৃতির নিয়মের সাথে নিজের মনকে একাকার করে প্রকৃতির স্বাদ নিজ উপলব্ধিতে আত্মস্ত করতে মরিয়া হয়ে উঠে।')

'সে কারণেই পর্যটকরা কুয়াকাটায় এসে প্রকৃতির নিয়মের সাথে নিজের মনকে একাকার করে প্রকৃতির স্বাদ নিজ উপলব্ধিতে আত্মস্ত করতে মরিয়া হয়ে উঠে।'

In [ ]:
stemmer.stem_word('কারণেই')

'কারণ'

In [34]:
!pip install bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.7 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=892f728b5ed26a6d0f0ff1261f311e8ee5c66f18724483e4b630f58929c2a69c
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: tqdm
    Found existi

In [35]:
from bnlp import CleanText

punkt not found. downloading...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [36]:
clean_text = CleanText(
   fix_unicode=True,
   unicode_norm=True,
   unicode_norm_form="NFKC",
   remove_url=False,
   remove_email=False,
   remove_emoji=False,
   remove_number=False,
   remove_digits=False,
   remove_punct=False,
   replace_with_url="<URL>",
   replace_with_email="<EMAIL>",
   replace_with_number="<NUMBER>",
   replace_with_digit="<DIGIT>",
   replace_with_punct = "<PUNC>"
)

input_text = "কারণেই"
clean_text = clean_text(input_text)
print(clean_text)

কারণেই


In [37]:
!pip install sbnltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.3 MB/s eta 0:00:00
  Created wheel for sbnltk: filename=sbnltk-2.0.2-py3-none-any.whl size=31767 sha256=ee9897bbe65b1a1c4e3b2749e5d2100a288669efad50910d1a422e10e2806786
  Stored in directory: /root/.cache/pip/wheels/bf/28/f7/edaa538b6d892af6d3a1ea6c622b62303658ce7928a413bcbe
Successfully built sbnltk


In [38]:
!git clone https://github.com/Foysal87/sbnltk.git

Cloning into 'sbnltk'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 122 (delta 58), reused 67 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (122/122), 51.61 KiB | 2.15 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [39]:
%cd sbnltk
!pip install .
%cd ../

/content/sbnltk
Processing /content/sbnltk
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sbnltk: filename=sbnltk-1.0.7-py3-none-any.whl size=31797 sha256=c80e6e8938ca59547a620559be55e3af442b40b0efb305db421143192d0da72a
  Stored in directory: /tmp/pip-ephem-wheel-cache-e8lo_xhu/wheels/79/0d/3d/af168498f543675a08fd12b7368710350be5fedb22242db060
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Successfully built sbnltk
Failed to build scikit-learn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (scikit-learn)
/content


In [40]:
from sbnltk.Stemmer import stemmerOP

Downloading...
From: https://drive.google.com/uc?id=142XvJg9xdpgzuYD31Y4pm-ZVdMaWmtuq
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/download_link.txt
100%|██████████| 1.66k/1.66k [00:00<00:00, 2.12MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UIR2arP_6Fm0MqJVbcG2Q9iLCnjQriDB
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/bangla_word_list.txt
100%|██████████| 15.4M/15.4M [00:00<00:00, 64.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1e4x5FQrdGyEfBBzQ_DOJ9wBxZZ1F7bx7
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/stopword_list.txt
100%|██████████| 4.77k/4.77k [00:00<00:00, 9.28MB/s]


In [41]:
st=stemmerOP()
print(st.stemWord('উপলব্ধিতে'))

Downloading...
From: https://drive.google.com/uc?id=1ohRYMGAeq4RZTa3W-7DXEsaUCaVNIxgO
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/rootword_list.txt
100%|██████████| 2.00M/2.00M [00:00<00:00, 78.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oBEK2DUs6Jt155AmMyc9565dHJJFZd_N
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/ner_static.txt
100%|██████████| 20.2M/20.2M [00:00<00:00, 56.7MB/s]


উপলব্ধিতে


In [ ]:
print(st.stemWord('প্রকৃতির'))

['র']
প্রকৃতি 2
Second return
প্রকৃতি


In [ ]:
print(st.stemSent('সে কারণেই পর্যটকরা কুয়াকাটায় এসে প্রকৃতির নিয়মের সাথে নিজের মনকে একাকার করে প্রকৃতির স্বাদ নিজ উপলব্ধিতে আত্মস্ত করতে মরিয়া হয়ে উঠে।'))

সে কারণ পর্যটক কুয়াকাটায় এস প্রকৃতি নিয়ম সাথ নিজ মন একাকা কর প্রকৃতি স্বাদ নিজ উপলব্ধিতে আত্মস্ত কর মরিয়া হ উঠে।


In [42]:
import os

In [ ]:
filenames = os.listdir("./Dataset")
print(filenames)

['porjotonlipi.bn']


In [ ]:
for filename in filenames:
    with open('./Dataset/' + filename, 'r') as file:
        current_line = ""
        previous_line = ""
        next_line = ""
        for line in file.readlines():
            next_line = st.stemSent(line.strip())
            print(next_line)

In [43]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-pn2jibwh
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-pn2jibwh
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6860 sha256=85ec2b362b37a2d2367921e0f419aaab9361cb080109cf80731e8b86a6b66e0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-vc2uzn9d/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186455 sha256=504cc0c710750b7d511d66ba345cb750f495410

In [44]:
from normalizer import normalize

In [ ]:
normalize("সে কারণেই পর্যটকরা কুয়াকাটায় এসে প্রকৃতির নিয়মের সাথে নিজের মনকে একাকার করে প্রকৃতির স্বাদ নিজ উপলব্ধিতে আত্মস্ত করতে মরিয়া হয়ে উঠে।")

'সে কারণেই পর্যটকরা কুয়াকাটায় এসে প্রকৃতির নিয়মের সাথে নিজের মনকে একাকার করে প্রকৃতির স্বাদ নিজ উপলব্ধিতে আত্মস্ত করতে মরিয়া হয়ে উঠে।'

In [ ]:
st.stemSent("উপলব্ধিতে")

'উপলব্ধিতে'

In [46]:
import re

In [ ]:
re.findall(r"কারণেই", "সে কারণেই পর্যটকরা কুয়াকাটায় এসে প্রকৃতির নিয়মের সাথে নিজের মনকে একাকার করে প্রকৃতির স্বাদ নিজ উপলব্ধিতে আত্মস্ত করতে মরিয়া হয়ে উঠে।")

['কারণেই']

In [45]:
!pip install pybmoore
import pybmoore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pybmoore: filename=pybmoore-2.0.2-cp310-cp310-linux_x86_64.whl size=132629 sha256=5996ebb1dd97d839668c13f59360f0c91e77f6fbb437817c035bbc29095402a1
  Stored in directory: /root/.cache/pip/wheels/a0/72/4b/a41342a9953085506f62bc761466088b56ee0b9e5462cd0d8e
Successfully built pybmoore


In [ ]:
mat = pybmoore.search('কারণেই', "সে কারণেই পর্যটকরা কুয়াকাটায় এসে প্রকৃতির নিয়মের সাথে নিজের মনকে একাকার করে প্রকৃতির স্বাদ নিজ উপলব্ধিতে আত্মস্ত করতে মরিয়া হয়ে উঠে।")

In [ ]:
len(mat)

1

In [49]:
file = open("./Dataset/porjotonlipi.bn", 'r')

In [50]:
for line in file:
    print(line)

Streaming output truncated to the last 5000 lines.
ট্রেকিং শুরু হয়েছিল তিরানা নামে একটি গ্রাম থেকে এবং আমরা যে গ্রামে (চেরাপুঞ্জি) থেকেছিলাম সেখান থেকে তিরানা প্রায় ২০ কিলোমিটার দূরে অবস্থিত।

তাই আমরা খুব তাড়াতাড়ি আমাদের যাত্রা শুরু করি এবং সকাল ৯টা টা নাগাদ তিরানাতে পৌঁছেছি।

খুব অল্প সংখ্যক মানুষ ঐদিন ওই পথে ট্র্যাকিং করেছিল।

পার্কিং লটে খুব কমই গাড়ি ছিল, আনুমানিক ৫ থেকে ৬ টি হবে।

সেখানকার ট্যুরিষ্ট গাইডরা পার্কিং এরিয়াতে ছিল, ডাবল ডেকার অবধি চার্জ ছিল ৬০০ রূপি এবং রেইনবো ফলস পর্যন্ত ছিল ১০০০ রূপি।

আমরা রেইনবো ফলস পর্যন্ত একজন গাইড নিয়েছিলাম।

যাত্রা শুরু করার পথে আমরা বহু গ্রামবাসীর সাথে দেখা আমাদের দেখা হয়, যারা আমাদের অভ্যর্থনা জানালেন।

ট্র্যাক থেকে ফিরে আসা কয়েকটি ব্যাগ প্যাকারকে দেখছিলাম এবং তারা স্পষ্টতই ক্লান্ত হয়ে পড়েছিল কারণ তারা উপরের দিকে উঠেছিল।

এক ঘন্টা পরে আমরা পাহাড়ের শেষ প্রান্তে পৌঁছে তারের ব্রিজটি পেরিয়ে যেতে হয়েছিল।

এই ব্রিজটির ধারণ ক্ষমতা বেশ ভালোই মনে হলো।

আর সেই নীচের নদীটি ছিল অসাধারণ।

আমরা বুঝতে পেরেছিলাম যে আমরা কিছুটা দেরি করে এগোচ্ছি আর সেই সাথে আম

In [51]:
file.name

./Dataset/porjotonlipi.bn


In [ ]:
suffix = {
    "কে": "high",
    'গুলোর': "medium",
    "েরটার": "low",
    "টার": "medium"
}

In [ ]:
sorted_suffix = dict(sorted(suffix.items(), key=lambda x: len(x[0]), reverse=True))
sorted_suffix

{'গুলোর': 'medium', 'েরটার': 'low', 'টার': 'medium', 'কে': 'high'}

In [ ]:
len(suffix)

4

In [ ]:
for i, k in sorted_suffix.items():
    print(i, k)

গুলোর medium
েরটার low
টার medium
কে high


In [52]:
!git clone https://github.com/Jayanta47/CEAT-Data-Collection.git

Cloning into 'CEAT-Data-Collection'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 254 (delta 155), reused 183 (delta 85), pack-reused 0 (from 0)
Receiving objects: 100% (254/254), 6.16 MiB | 5.65 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [53]:
!pip install pybmoore

In [ ]:
!python ./CEAT-Data-Collection/dataScrapper.py ./CEAT-Data-Collection/sample_data/

পর্যটনলিপি আজ আপনা সামন তুলে ধরবে ঢাক বু অবস্থিত এক ঐতিহাসিক ভবন কার্জন হল।
এ নির্মিত হ ঢাকা কলেজ পাঠাগার হিসাব এবং নির্মাণ জন্য অর্থ প্রদান কর ভাওয়াল রাজকুমার।
১৯০৪ সাল ঢাকা প্রকাশ লিখেছিল_ "ঢাকা কলেজ নিমতলী স্থানান্তরিত হইবে।
এই কলেজ সংশ্রবে এক পাঠাগার নির্মাণ জন্য সুযোগ্য প্রিন্সিপাল ডাক্তার রা মহাশয় যত্নবান ছিলেন।
১৯১১ সাল বঙ্গভঙ্গ রদ হলে, ঢাকা কলেজ ক্লাস নেয়া হ থা কার্জন হলে।
({'প্রকৃতি': [], 'পর্যটনলিপি': [0], 'কুয়াকাটা': [], 'বৌদ্ধ': [], 'লালবাগ': [], 'কলেজ': [1, 2, 3, 4]}, ['কার্জন হল ও ত সংক্ষিপ্ত ইতিহাস। পর্যটনলিপি আজ আপনা সামন তুলে ধরবে ঢাক বু অবস্থিত এক ঐতিহাসিক ভবন কার্জন হল। এ ঢাকা বিশ্ববিদ্যাল এলাকা অবস্থিত।', 'কিন্তু শরীফউদ্দীন আহমদ এক প্রবন্ধ দেখিয় এ ধারণা ভুল। এ নির্মিত হ ঢাকা কলেজ পাঠাগার হিসাব এবং নির্মাণ জন্য অর্থ প্রদান কর ভাওয়াল রাজকুমার। ১৯০৪ সাল ঢাকা প্রকাশ লিখেছিল_ "ঢাকা কলেজ নিমতলী স্থানান্তরিত হইবে।', 'এ নির্মিত হ ঢাকা কলেজ পাঠাগার হিসাব এবং নির্মাণ জন্য অর্থ প্রদান কর ভাওয়াল রাজকুমার। ১৯০৪ সাল ঢাকা প্রকাশ লিখেছিল_ "ঢাকা কলেজ নিমতলী স্থানান্তরিত হইবে। এই কল

In [54]:
import re

In [55]:
with open("porjontolipi.bn", "r") as file:
    lines = file.read().split("\n")
    string = " ".join(lines)

print(string)

যমুনা পাড়ের গল্প বন্ধুরা আজ পর্যটনলিপি আপনাদের সামনে তুলে ধরবে যমুনা নদীর তীরবর্তী জেলা সিরাজগঞ্জ। সিরাজগঞ্জ জেলায় বঙ্গবন্ধু যমুনা বহুমূখী সেতু, হার্ডপয়েন্ট, রাণীগ্রাম গ্রোয়েন, কাঁটাখাল, ইলিয়ট ব্রীজ জেলার ঐতিহ্য বহন করে চলেছে। বঙ্গবন্ধু যমুনা বহুমূখী সেতুর (বাংলাদেশের বৃহত্তম সড়ক ও রেলসেতু) পাশাপাশি সয়দাবাদ ইউনিয়নে বঙ্গবন্ধু যমুনা বহুমূখী সেতুসংলগ্ন একটি 'ইকোপার্ক' প্রতিষ্ঠা করা হলেও জনগণের চাহিদানুযায়ী এখনও গড়ে উঠেনি। পরিপূর্ণভাবে গড়ে উঠলে দেশী-বিদেশী পর্যটকদের ব্যাপক সমাগম ঘটবে মর্মে আশা করা যায়। এছাড়া পৌর এলাকার হার্ডপয়েন্টে একটি পার্ক তৈরী করা হয়েছে। আবাসনের জন্য ইতোমধ্যেই শহর এলাকায় গড়ে উঠেছে দু'তিনটি উন্নতমানের আবাসিক হোটেল। উল্লাপাড়ার ঘাটিনা ব্রীজ সংলগ্ন এলাকায় প্রতিদিন বিকেলে আনন্দ ভ্রমণের জন্য লোকজন জড়ো হয়। এছাড়া বর্ষাকালে চলনবিল বেষ্টিত মোহনপুর, উধুনিয়া, বড়পাঙ্গাসী ও বাঙ্গালা ইউনিয়নের বিস্তীর্ণ অঞ্চল নয়নাভিরাম শোভা ধারণ করে। সুফি সাধক শাহ কামাল (রহঃ) ধর্ম প্রচারের জন্য কামারখন্দে আসেন এবং তিনি ভদ্রঘাট ইউনিয়নের নান্দিনা কামালিয়া গ্রামে মৃত্যু বরণ করেন। তাঁর কবরকে ঘিরে তৈরী হয়ে

In [56]:
re.match(r'\bগান(?:টি|গুলো|গুলি)?\W', "গানগুলোর বাগান  ")

In [57]:
re.match(r'\bমা(?:টি|গুলো|গুলি|রা|য়েরা|র)?\W', "শাপলার সিজন: শাপলা হয় প্রায় ৯ মাস। মার্চ থেকে শুরু করে নভেম্বর পর্যন্ত শাপলার পুরো সিজন। তাই যারা শাপলা বিলের এই মনোরম দৃশ্য উপভোগ করতে চান তারা এই মার্চ থেকে নভেম্বরের মধ্যে চলে যেতে পারেন সাতলা গ্রামে।")

In [58]:
suffixList = [
'কে',
'তে',
'তো',
'টি',
'টির',
'েরটা',
'েরটার',
]

In [59]:
suffixString = '|'.join(suffixList)

In [60]:
f"(?:{suffixString})?"

'(?:কে|তে|তো|টি|টির|েরটা|েরটার)?'

In [61]:
string = "ঢাকের সদরঘাট থেকে বিলাস বহুল ডাবল ডেকার এম.ভি পারাবত, এম.ভি সৈকত, এম.ভি সুন্দরবন"

In [62]:
suffixList = [
"টি", "টির", "টা", "টার", "গুলো", "গুলি", "গুলোর", "ের"
]
suffixString = '|'.join(suffixList)
f"(?:{suffixString})?"

'(?:টি|টির|টা|টার|গুলো|গুলি|গুলোর|ের)?'

In [63]:
r'\b' + re.escape("ঢাক")  + r'[ ,।:@;\'!#$%\^&\~\-\+?><\(\)]'

"\\bঢাক[ ,।:@;\\'!#$%\\^&\\~\\-\\+?><\\(\\)]"

In [ ]:
re.match(r'\b' + re.escape("ঢাক")  + r'[ ,।:@;\'!#$%&\~\-\+?><\(\)]', string)

In [64]:
import re

In [65]:
main_pattern = "ঢাক"
end_characters = r' ,।:@;\'!#$%\^&\~\-\+?><\(\)'

regex_pattern = r'\b' + re.escape(main_pattern) + f"(?:{suffixString})?"+ r'[' + re.escape(end_characters) + r']'

In [66]:
regex_pattern

"\\bঢাক(?:টি|টির|টা|টার|গুলো|গুলি|গুলোর|ের)?[\\ ,।:@;\\\\'!\\#\\$%\\\\\\^\\&\\\\\\~\\\\\\-\\\\\\+\\?><\\\\\\(\\\\\\)]"

In [ ]:
re.match(regex_pattern, string)

<re.Match object; span=(0, 6), match='ঢাকের '>

In [ ]:
r'\b' + re.escape("ঢাক") + f"(?:{suffixString})?" + r'\W'

In [67]:
import re

In [ ]:
def find_words_with_pattern_in_string(input_string, pattern):
    matched_words = re.findall(r'\b' + pattern + r'\b', input_string)
    return matched_words

input_string = "কেল্লার চত্বরে তিনটি স্থাপনা রয়েছে কেন্দ্রস্থলের দরবার হল ও হাম্মাম খানা, পরীবিবির সমাধি, উত্তর পশ্চিমাংশের শাহী মসজিদ। এছাড়া দক্ষিণ-পূর্বাংশে সুদৃশ্য ফটক, এবং দক্ষিণ দেয়ালের ছাদের উপরে বাগান রয়েছে। বর্তমানে রবিবার পূর্ণ দিবস ও সোমবার অর্ধদিবস বন্ধ থাকে।"
pattern = "গান"

matched_words = find_words_with_pattern_in_string(input_string, pattern)
print("Matched Words:")
if matched_words:
    for word in matched_words:
        print(word)

Matched Words:
গান


In [ ]:
pattern = r"বিশ্ববিদ্যালয়"

with open("porjotonlipi.bn", "r") as file:
    for i, line in enumerate(file):
        line = line.strip()
        matched_words = find_words_with_pattern_in_string(line, pattern)
        if len(matched_words):
            print(i, line)

2 এটি ঢাকা বিশ্ববিদ্যালয় এলাকায় অবস্থিত।
4 এটি বর্তমানে ঢাকা বিশ্ববিদ্যালয়ের বিজ্ঞান ও জীব বিজ্ঞান অণুষদের কিছু শ্রেনীকক্ষ ও পরীক্ষার হল হিসেবে ব্যবহৃত হচ্ছে।
14 পরবর্তী সময়ে ১৯২১ সালে ঢাকা বিশ্ববিদ্যালয় স্থাপিত হলে কার্জন হল অন্তর্ভুক্ত হয় ঢাকা বিশ্ববিদ্যালয়ের বিজ্ঞান বিভাগের জন্য, যা আজও ব্যবহৃত হচ্ছে।


## Reduced dataset embedding extracting


In [68]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import pickle
import json
from tqdm import tqdm

In [69]:
%%capture
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
from normalizer import normalize

In [ ]:
from transformers import AutoModelForPreTraining

In [70]:
model2 = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert_large")
tokenizer2 = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large")
model2.eval()

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

ElectraForPreTraining(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-23): 24 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): 

In [71]:
def normalizeWeatDict(weatWordDict):
    newWeatWordDict = {}
    for word in weatWordDict:
        normalizedWord = normalize(
            word, unicode_norm="NFKC", apply_unicode_norm_last=True
        )
        newList = []
        for suffix in weatWordDict[word]:
            normalizedSuffix = normalize(
                suffix, unicode_norm="NFKC", apply_unicode_norm_last=True
            )
            newList.append(normalizedSuffix)
        newWeatWordDict[normalizedWord] = newList
    return newWeatWordDict

In [ ]:
weatWordDict = json.load(open("weatWordsWithSuffix.jsonl", "r", encoding="utf-8"))
weatWordDict = normalizeWeatDict(weatWordDict)

In [73]:
from abc import ABC, abstractmethod
import re
from normalizer import normalize

In [75]:
class WordFinder(ABC):
    def __init__(self, weatWordList: list[str]):
        self.weatWordList = weatWordList
        self.weatWordDict = {}
        for word in weatWordList:
            self.weatWordDict[word] = []

    def getWeatWordDict(self):
        return self.weatWordDict

    @abstractmethod
    def evaluate(self, sent: str, serial: int) -> bool:
        pass

    @abstractmethod
    def getIndex(self, sentence, keyWord) -> list[int]:
        pass

    @abstractmethod
    def getSpan(self, sent: str, keyWord: str) -> list[int]:
        pass

    @abstractmethod
    def getSpanByIndex(self, sent: str, keyWord: str, index: int) -> list[int]:
        pass


class WordEvaluatorRegexSuffixFixed(WordFinder):
    def __init__(self, weatWordDict: dict[str, list[str]]):
        super().__init__(weatWordDict.keys())
        self.weatWordPatterns = {}
        end_characters = r" ,।:@;\'\"!#\$%\^&\~\-\+\?><\(\)"
        for word in weatWordDict:
            suffixString = "|".join(weatWordDict[word])
            self.weatWordPatterns[word] = (
                r"\b"
                + re.escape(word)
                + f"(?:{suffixString})?"
                + r"["
                + re.escape(end_characters)
                + r"]"
            )

    def evaluate(self, sent: str, serial: int) -> bool:
        isMatchFound = False
        for word in self.weatWordList:
            pattern = self.weatWordPatterns[word]
            matches = re.match(pattern, sent)
            if matches:
                isMatchFound = True
                self.weatWordDict[word].append(serial)
        return isMatchFound

    def getIndex(self, sentence, keyWord) -> list[int]:
        pattern = self.weatWordPatterns[keyWord]
        sentence = normalize(sentence)
        indices = []
        for i, word in enumerate(sentence.split()):
            if re.match(pattern, word) or re.match(pattern, word + " "):
                indices.append(i)
        return indices

    def getSpan(self, sent: str, keyWord: str):
        pattern = self.weatWordPatterns[keyWord]
        normalizedSent = normalize(sent)
        firstWord = normalizedSent.split()[0]
        if re.match(pattern, firstWord) or re.match(pattern, firstWord+" "):
            return 0, len(keyWord)
        matches = re.search(pattern=" "+keyWord, string=normalizedSent)
        if matches:
            start, end = matches.span()
            return start+1, end
        else:
            return re.search(pattern=pattern, string=normalizedSent).span()

    def getSpanByIndex(self, sent:str, keyWord: str, index: int):
        currentPos = 0;
        normalizedSent = normalize(sent)
        for i, word in enumerate(normalizedSent.split()):
            if i == index:
                return currentPos, currentPos + len(keyWord)
            else:
                currentPos += len(word)+1

In [ ]:
evaluator = WordEvaluatorRegexSuffixFixed(weatWordDict)

In [77]:
from abc import ABC, abstractmethod
from normalizer import normalize
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [78]:
class ModelWrapper(ABC):
    def setEmbeddingLayer(self, layer: int) -> None:
        self.layer = layer

    @abstractmethod
    def getWordVector(
        self, word: str, sent: str, index: int, span: list[int]
    ) -> np.array:
        pass


class MLMEmbeddingExtractor(ModelWrapper):
    def __init__(self, model_name: str, tokenizer_name: str) -> None:
        self.model = AutoModelForMaskedLM.from_pretrained(
            model_name, output_hidden_states=True
        )
        self.tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_name
        )

        if torch.cuda.is_available():
            self.model = self.model.to("cuda")

        self.layer = 24

    def getTokenIndices(self, offset_mapping, index, span):
        indices = []
        for i, token in enumerate(offset_mapping):
            if i < index or i == 0:
                continue
            if token[0] >= span[0] and token[0] < span[1]:
                indices.append(i)
            elif token[0] > span[1]:
                break
        assert len(indices) > 0
        return indices

    def prepareEmbedding(self, output, tokenIndices, averagePooling=True):
        if averagePooling:
            return np.mean(
                output[1][self.layer][0].detach().cpu().numpy()[tokenIndices], axis=0
            )
        else:
            return np.max(
                output[1][self.layer][0].detach().cpu().numpy()[tokenIndices], axis=0
            )

    def getWordVector(
        self, word: str, sent: str, index: int, span: list[int]
    ) -> np.array:
        normalized_sentence = normalize(sent)

        input_tokens = self.tokenizer(normalized_sentence, return_tensors="pt")

        input_token_offsets = self.tokenizer(
            normalized_sentence,
            return_offsets_mapping=True,
            return_tensors="pt",
        ).offset_mapping[0]

        token_indices = self.getTokenIndices(input_token_offsets, index, span)
        if torch.cuda.is_available():
            input_tokens = input_tokens.to("cuda")
        with torch.no_grad():
            output = self.model(**input_tokens)

            return self.prepareEmbedding(output, token_indices)


In [79]:
modelMurilBase = MLMEmbeddingExtractor(
        model_name="google/muril-base-cased",
        tokenizer_name="google/muril-base-cased",
    )
modelMurilBase.setEmbeddingLayer(12)

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [80]:
modelXLMRobertaBase = MLMEmbeddingExtractor(
        model_name="xlm-roberta-base",
        tokenizer_name="xlm-roberta-base",
    )
modelXLMRobertaBase.setEmbeddingLayer(12)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [81]:
def getSentencesSample(sentenceList, maxItems=1000):
    numItems = min(maxItems, len(sentenceList))
    items = (
        sentenceList if numItems < maxItems else random.sample(sentenceList, maxItems)
    )
    return items


class SentenceProcessor:
    def __init__(self, evaluator: WordFinder, stemmer = None) -> None:
        self.evaluator = evaluator
        self.stemmer = stemmer

    def setLength(self, length: int) -> None:
        self.length = length

    def shortenSentence(self, sentence: str, word: str) -> str:
        indices = self.evaluator.getIndex(sentence, word)
        if len(indices) == 0:
            raise Exception("No match found")

        index = indices[0]
        if self.length == -1:
            return sentence, index
        words = sentence.split()
        modifiedIndex = index
        if len(words) >= self.length:
            if index < self.length // 2:
                wordsUsed = words[: self.length]
            elif (len(words) - index) < self.length // 2 + 1:
                wordsUsed = words[-self.length :]
                modifiedIndex = self.length - (len(words) - index)
            else:
                wordsUsed = words[index - self.length // 2 : index + self.length // 2]
                modifiedIndex = self.length // 2
            newSentence = " ".join(wordsUsed)
        else:
            newSentence = sentence

        if self.stemmer:
            newSentence = self.stemmer.stemSentence(newSentence)
        return newSentence, modifiedIndex

    def getSpan(self, sentence: str, word: str, index: int) -> list[int]:
        return self.evaluator.getSpanByIndex(sentence, word, index)


class EmbeddingExtractor:
    def __init__(
        self, sentenceProcessor: SentenceProcessor, model: ModelWrapper
    ) -> None:
        self.sentenceProcessor = sentenceProcessor
        self.model = model

    def savePickle(self, filename: str, data: dict[str, list[np.array]]):
        pickle.dump(data, open(filename, "wb"))

    def extract(
        self, weatWordSentenceDict: dict[str, list[str]]
    ) -> dict[str, list[np.array]]:
        weatWordEmbeddings = {}

        for word in weatWordSentenceDict:
            weatWordEmbeddings[word] = []

        for word in weatWordSentenceDict:
            print(f"\nCurrent Word: {word}")
            for index, sentence in tqdm(
                enumerate(weatWordSentenceDict[word]), desc="Processing Sentences"
            ):
                sentence, index = self.sentenceProcessor.shortenSentence(sentence, word)
                print(sentence, index)
                span = self.sentenceProcessor.getSpan(sentence, word, index)

                weatWordEmbeddings[word].append(
                                        self.model.getWordVector(word, sentence, index, span)
                                    )

        return weatWordEmbeddings

In [ ]:
processor = SentenceProcessor(evaluator)
processor.setLength(15)

extractor = EmbeddingExtractor(processor, modelXLMRobertaBase)

In [ ]:
weatWordSentenceDict = pickle.load(open("./results_small.pkl", "rb"))
new_dict =  {}
new_dict["হারমোনিয়াম"] = weatWordSentenceDict["হারমোনিয়াম"][:10]
embedding = extractor.extract(weatWordSentenceDict)


Current Word: মা


Processing Sentences: 0it [00:00, ?it/s]

অপুষ্টিজনিত কারনে কংকালসার হয়ে পড়ে শিশু। মায়ের দুধ খাওয়ানকে পবিত্র কোরআনে ওয়াজিব বলা হয়েছে। মা 6


Processing Sentences: 1it [00:00,  2.03it/s]

মায়ের দুধ খাওয়ানকে পবিত্র কোরআনে ওয়াজিব বলা হয়েছে। মা হেপাটাইটিস বি-ভাইরাস সহ যেকোন সংক্রামক ব্যাধিতে 0


...

Processing Sentences: 8it [00:03,  2.02it/s]

translation of Lilavati আরও দেখুন. ভারতীয় গণিত বীজগণিত এবং জ্যামিতি সময়রেখা বহিঃসংযোগ. ভাষ্করাচার্যের লীলাবতী-মূল 7


Processing Sentences: 9it [00:04,  1.99it/s]

প্যারাগ্রাফ রাইটিং, লেটার রাইটিং ইত্যাদি থাকতে পারে। বীজগণিত ও পাটিগণিত অংশ থেকে প্রশ্ন করা 7


Processing Sentences: 10it [00:04,  2.05it/s]


In [ ]:
print(type(embedding["হারমোনিয়াম"][0]))

<class 'numpy.ndarray'>


In [ ]:
embedding["হারমোনিয়াম"][0].shape

(1024,)

## Post processing and eliminating

In [82]:
from google.colab import drive

In [83]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/ceat_bn_data/results_final.pkl ./results.pkl

In [84]:
import pickle

In [ ]:
weatWordSentenceDict = pickle.load(open("./results.pkl", "rb"))

In [ ]:
type(weatWordSentenceDict)

dict

In [ ]:
sorted_keys = sorted(
        weatWordSentenceDict.keys(), key=lambda k: len(weatWordSentenceDict[k]), reverse=True
    )

In [ ]:
for key in sorted_keys:
    print(f"{key}: {len(weatWordSentenceDict[key])}")

In [ ]:
f = open("./all_weat_words.txt", "r")
wordList = []
for word in f.readlines():
    word = normalize(
            word.strip(), unicode_norm="NFKC", apply_unicode_norm_last=True
        )
    wordList.append(word)

wordList = sorted(wordList)

for word in wordList:
    print(f"{word}: {len(weatWordSentenceDict[word])}")

In [ ]:
w = open("./all_weat_words.txt", "w")

for word in wordList:
    w.write(word+"\n")

w.close()

In [ ]:
def merge_same_words(
    word_sentence_dict: dict[str, list[str]], same_words_dict: dict[str, list[str]]
) -> dict[str, list[str]]:
    for masterWord, value in same_words_dict.items():
        new_list = []
        for word in value:
            if word in word_sentence_dict:
                old_list = word_sentence_dict[word]
                for sent in old_list:
                    new_sent = re.sub(word, masterWord, sent)
                    new_list.append(new_sent)
                word_sentence_dict.pop(word)
        word_sentence_dict[masterWord].extend(new_list)
    return word_sentence_dict

In [86]:
import re

In [ ]:
weatWordSentenceDict = merge_same_words(weatWordSentenceDict, {
    "পদার্থবিজ্ঞান": ["পদার্থবিদ্যা"],
})

In [ ]:
len(weatWordSentenceDict)

262

In [ ]:
unwanted = []
for word in weatWordSentenceDict:
    if word not in wordList:
        unwanted.append(word)

In [ ]:
unwanted

In [ ]:
for word in unwanted:
    weatWordSentenceDict.pop(word)

In [ ]:
len(weatWordSentenceDict)

216

In [ ]:
pickle.dump(weatWordSentenceDict, open("result_final.pkl", "wb"))

In [ ]:
!cp ./result_final.pkl /content/drive/MyDrive/ceat_bn_data/result_final.pkl

## CEAT dataset embedding extracting

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [91]:
%%capture
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer

In [92]:
!git clone https://github.com/Jayanta47/CEAT-Data-Collection.git

fatal: destination path 'CEAT-Data-Collection' already exists and is not an empty directory.


In [ ]:
!pip install pybmoore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pybmoore: filename=pybmoore-1.4.0-cp310-cp310-linux_x86_64.whl size=130746 sha256=a27aa15538c29f27bd76c4cd96de801b46c78fb3bd247a50214c72cdb1278cb7
  Stored in directory: /root/.cache/pip/wheels/c2/0c/c7/85e8f68afcce4790b7dd50101aeee7be677da7554a6658a01c
Successfully built pybmoore


In [93]:
%cd /content/CEAT-Data-Collection

/content/CEAT-Data-Collection


In [ ]:
!mkdir ./results
!cp /content/drive/MyDrive/ceat_bn_data/results_final.pkl ./results/results.pkl

In [ ]:
!mkdir ./embeddings

In [ ]:
!python extractEmbeddings.py -l 9

2023-09-23 23:33:52.983574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Processing For: গোলাপ
Processing Sentences: 4562it [02:02, 37.14it/s]
Processing For: জবা
Processing Sentences: 1185it [00:30, 38.97it/s]
Processing For: শাপলা
Processing Sentences: 1125it [00:28, 32.74it/s]^C


In [ ]:
!rm -rf /content/CEAT-Data-Collection

## Trait sentence generating

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/ceat_bn_data/V2/Trait_Results.zip ./
!unzip ./Trait_Results.zip

Archive:  ./Trait_Results.zip
   creating: Trait_Results/
  inflating: Trait_Results/results_trait.pkl  
  inflating: Trait_Results/results_trait.txt  


In [94]:
%%capture
!pip install git+https://github.com/csebuetnlp/normalizer

In [95]:
import pickle
from normalizer import normalize

In [ ]:
traitWords = pickle.load(open("/content/Trait_Results/results_trait.pkl", "rb"))
with open("/content/allPositiveWords.txt", "r") as f:
    words = f.readlines()
    positiveWords = [x.strip() for x in words]
    positiveWords = [normalize(x) for x in positiveWords]

In [ ]:
positiveTraitDict = {}
for positiveWord in positiveWords:
    positiveTraitDict[positiveWord] = traitWords[positiveWord]

In [ ]:
len(positiveTraitDict.keys())

109

In [97]:
import random

In [ ]:
!mkdir ./samples
for word, sentences in positiveTraitDict.items():
    samples = random.sample(sentences, len(sentences))
    with open(f"./samples/{word}.txt", "w") as f:
        for sentence in samples:
            f.write(sentence+"\n")
        f.close()

In [ ]:
!zip -r samples.zip ./samples

  adding: samples/ (stored 0%)
  adding: samples/অভেদ্য.txt (deflated 73%)
  adding: samples/প্রশংসনীয়.txt (deflated 77%)
  adding: samples/গতিশীল.txt (deflated 77%)
  adding: samples/ক্ষমাশীল.txt (deflated 75%)
  adding: samples/নম্র.txt (deflated 75%)
  adding: samples/সংবেদনশীল.txt (deflated 78%)
  adding: samples/সূক্ষ্ম.txt (deflated 75%)
  adding: samples/দুঃসাহসিক.txt (deflated 77%)
  adding: samples/কর্তব্যপরায়ণ.txt (deflated 75%)
  adding: samples/সরল.txt (deflated 76%)
  adding: samples/নান্দনিক.txt (deflated 77%)
  adding: samples/বিবেচ্য.txt (deflated 78%)
  adding: samples/সৃজনশীল.txt (deflated 82%)
  adding: samples/প্রেমময়.txt (deflated 75%)
  adding: samples/ব্যক্তিত্বপূর্ণ.txt (deflated 78%)
  adding: samples/কমনীয়.txt (deflated 73%)
  adding: samples/শক্তিশালী.txt (deflated 77%)
  adding: samples/আশাবাদী.txt (deflated 76%)
  adding: samples/সামাজিক.txt (deflated 78%)
  adding: samples/অদ্বিতীয়.txt (deflated 77%)
  adding: samples/শান্তিময়.txt (deflated 77%)
  ad

In [ ]:
word = normalize("আবেগপ্রবণ")
with open(f"./samples/{word}.txt") as f:
    lines = f.readlines()
    for line in random.sample(lines, min(20, len(lines))):
        print(line)

বড় ধরনের লেনদেনের সময় কাউকে সঙ্গে রাখুন ও দলিল করে নিতে ভুলবেন না। আবেগপ্রবণ কোনো খবরে আজ আপনি উদ্বেলিত হয়ে উঠতে পারেন। কন্যা : ২২ আগস্ট-২১ সেপ্টেম্বর

আমি একবার পাথর হয়েছিলাম। আবেগপ্রবণ হয়ে যখন আপন হই, আপনজনরা তখন অপমান করে।' আনীল... 'কবর সামনে। মৃত্যুর সাথে বীর কুস্তি ধরে।

আবেগ কি মানুষকে বোকা বানায়? আবেগপ্রবণ মানুষেরা নিজেদের বোকা ভাবে। বিশেষ করে আঘাত পেলে।

কোনোভাবে নিজেকে হারিয়ে ফেলেনি তো? আবেগপ্রবণ, বড়ো বেশি আবেগপ্রবণ। কীভাবে আছে, আগে থেকে অনুমান করা খুব কঠিন।

কান্না মনের কষ্ট দূর করে মন হালকা করে থাকে। আবেগপ্রবণ কান্নাতে এনডোরফিন নামক উপাদান নিঃসৃত হয় যা দুশ্চিন্তা দূর করে থাকে। তাই মন খারাপের সময় কাঁদুন, দেখবেন মন অনেকখানি ভালো হয়ে গেছে।

কিন্তু ডা. জাফরুল্লাহ সাহেব বাংলাদেশে র‍্যাপিড কিট ব্যবহারের জন্য এক প্রকার জিহাদ ঘোষণা করেছেন। আবেগপ্রবণ কিছু মানুষ বলছে, উনাকে সুযোগ দিলে কি হয়? দেশিদের কদর আমরা দিতে পারিনা ইত্যাদি ইত্যাদি ।

তাদের অত্যন্ত কেতাদুরস্ত ভাবটা প্রকাশ পেত হাতের লাঠি ও হাঁটার ভঙ্গিতে; অনেকটা ইংল্যান্ডের স্যান্ডহার্স্ট শহরের অধিবাসীদের মত। আবেগপ্রবণ 